In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
import shutil

from hloc import (
    extract_features,
    match_features,
    match_dense,
    reconstruction,
    visualization,
    pairs_from_all,
    localize_sfm,
    pairs_from_exhaustive
)

ROOT = Path().absolute()
# add root to path in order to execute scripts
if str(ROOT) not in sys.path:
  sys.path.append(str(ROOT))

In [ ]:
DATASET = Path("datasets/framlingham2")

IMAGES = DATASET / "images"
QUERY_IMAGES = DATASET / "query"
QUERY_IMAGE = QUERY_IMAGES / "img.png" # img that will be localized

OUT = Path("outputs/sfm/")
SFM_PAIRS = OUT / "pairs.txt"
SFM_DIR = OUT / "framlingham_s2d+nn"

# retrieval_conf = extract_features.confs["netvlad"]
feature_conf = extract_features.confs["s2dnet"]
matcher_conf = match_features.confs["NN-ratio"]
# matcher_conf = match_dense.confs["loftr"]

In [ ]:
pairs_from_all.main(SFM_PAIRS, IMAGES)

In [ ]:
feature_path = extract_features.main(feature_conf, IMAGES, SFM_DIR)
#feature_path = Path(outputs, feature_conf["output"] + ".h5")

In [ ]:
match_path = match_features.main(
    conf=matcher_conf,
    pairs=SFM_PAIRS,
    features=feature_conf["output"],
    export_dir=OUT,
)

"""
feature_path, match_path = match_dense.main(
  conf=matcher_conf, 
  pairs=sfm_pairs, 
  image_dir=images, 
  export_dir=outputs,
  features_ref=feature_path
)
"""

## 3D reconstruction
Run COLMAP on the features and matches.

In [ ]:
model = reconstruction.main(SFM_DIR, IMAGES, SFM_PAIRS, feature_path, match_path)

In [ ]:
from third_party.Neuralangelo.convert_data_to_json import data_to_json

args = {
  "data_dir": str(SFM_DIR.absolute()),
  "scene_type": "outdoor",
  "image_dir": str(IMAGES.absolute()),
}

data_to_json(args)

SFM_IMAGES = SFM_DIR / "images"
if not SFM_IMAGES.exists():
  # copy files from input images to self contained project path
  # os.mkdir(new_input_path)
  print("copied input images to self contained SfM reconstruction folder")
  shutil.copytree(IMAGES, SFM_IMAGES)

In [ ]:
"""
localize_sfm.main(
    sfm_dir,
    dataset / "queries/*_time_queries_with_intrinsics.txt",
    loc_pairs,
    features,
    loc_matches,
    results,
    covisibility_clustering=False,
)  # not required with SuperPoint+SuperGlue
"""

# https://colab.research.google.com/drive/1MrVs9b8aQYODtOGkoaGNF9Nji3sbCNMQ#scrollTo=74f07f86

# register new QUERY_IMAGE
references_registered = [model.images[i].name for i in model.reg_image_ids()]
extract_features.main(
    feature_conf,
    IMAGES,
    image_list=QUERY_IMAGES,
    feature_path=feature_path,
    overwrite=True,
)
pairs_from_exhaustive.main(
    SFM_PAIRS, image_list=QUERY_IMAGES, ref_list=references_registered
)
match_features.main(
    matcher_conf, SFM_PAIRS, features=feature_path, matches=match_path, overwrite=True
)

import pycolmap
from hloc.localize_sfm import QueryLocalizer, pose_from_cluster

camera = pycolmap.infer_camera_from_image(QUERY_IMAGE)
ref_ids = [model.find_image_with_name(n).image_id for n in references_registered]
conf = {
    "estimation": {"ransac": {"max_error": 12}},
    "refinement": {"refine_focal_length": True, "refine_extra_params": True},
}
localizer = QueryLocalizer(model, conf)
ret, log = pose_from_cluster(
    localizer, QUERY_IMAGE, camera, ref_ids, feature_path, match_path
)

print(f'found {ret["num_inliers"]}/{len(ret["inliers"])} inlier correspondences.')
visualization.visualize_loc_from_log(IMAGES, QUERY_IMAGE, log, model)

## Visualization
We visualize some of the registered images, and color their keypoint by visibility, track length, or triangulated depth.

In [ ]:
#visualization.visualize_sfm_2d(model, images, color_by="visibility", n=5)

In [ ]:
#visualization.visualize_sfm_2d(model, images, color_by="track_length", n=5)

In [ ]:
#visualization.visualize_sfm_2d(model, images, color_by="depth", n=5)